In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
from sklearn.impute import SimpleImputer

In [ ]:
#example_sample_submission = pd.read_csv('/kaggle/input/jane-street-market-prediction/example_sample_submission.csv')
#features = pd.read_csv("/kaggle/input/jane-street-market-prediction/features.csv")
#example_test = pd.read_csv("/kaggle/input/jane-street-market-prediction/example_test.csv")
train = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv")

In [ ]:
FEATURES = [col for col in list(train.columns) if 'feature' in col]

In [ ]:
train = train[train['weight'] != 0]
train['action'] = (train['resp'] > 0).astype(int)
sns.countplot(train['action'])

In [ ]:
missing_values = pd.DataFrame()
missing_values['column'] = FEATURES
missing_values['num_missing'] = [train[i].isna().sum() for i in FEATURES]

missing_values.T


In [ ]:
#example_sample_submission.head(5)

In [ ]:
train.head(5)

In [ ]:

train = train[train['weight'] != 0]
train['action'] = (train['resp'].values > 0).astype(int)

train = train.fillna(-999999)
X_train = train.loc[:, FEATURES]
y_train = train.loc[:, 'action']
del train

In [ ]:
X_train[:3]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import xgboost as xgb
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

In [ ]:
#clfs = [CatBoostClassifier(learning_rate=0.5, iterations=20),  LGBMClassifier()]
clfs = [xgb.XGBClassifier(
    n_estimators=1000,
    max_depth=7,
    eta=0.1,
    missing=-999999,
    tree_method='gpu_hist',
    eval_metric='error'
)]

In [ ]:
def train_clfs(clfs, train_X, train_y):
    for i, clf in enumerate(clfs):
        print(i)
        clf.fit(train_X, train_y)
        
    return clfs

In [ ]:
clfs = train_clfs(clfs, X_train, y_train)

In [ ]:
import janestreet
env = janestreet.make_env()

In [ ]:
iter_test = env.iter_test() 

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    wt = test_df.iloc[0].weight
    if(wt == 0):
        sample_prediction_df.action = 0 
    else:
        sample_prediction_df.action = clfs[0].predict(test_df.loc[:, FEATURES])
                                                     
    env.predict(sample_prediction_df)